Authenticating user profile to access google drive data

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloading the dataset as emotion.zip

In [0]:
#downloaded = drive.CreateFile({'id':'1FUZJECbq2GSXnzoBXpptz0-3IapCXUKx'}) # replace the id with id of file you want to access
#downloaded.GetContentFile('emotion.zip')

unzipping the dataset

In [0]:
#%%capture
#!unzip emotion.zip

In [4]:
!ls

adc.json  emotion.zip  meld  sample_data


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Loading the required libraries

In [0]:
import librosa
import librosa.display
import os, glob, pickle
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest, GenericUnivariateSelect, SelectFromModel, VarianceThreshold
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_recall_fscore_support

In [0]:
import random
from scipy.io import wavfile
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MaxAbsScaler
from collections import Counter
from imblearn.over_sampling import SMOTE

Giving numeric class labels to the text class names

In [0]:
d = {'disgust':2, 'fear':4, 'happy':1, 'neutral':0, 'sad':3}

function to extract features from a single sound file

In [0]:
def sound2features(filename):
    sr, y = wavfile.read(filename)
    y = y.astype('float32')
    y = np.mean(y,axis=1)/32767
    mfcc = librosa.feature.mfcc(
                y=y, sr=sr, hop_length=512)
    spectral_center = librosa.feature.spectral_centroid(
                y=y, sr=sr, hop_length=512)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
    spectral_contrast = librosa.feature.spectral_contrast(
                y=y, sr=sr, hop_length=512)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr, hop_length=512)
    flatness = librosa.feature.spectral_flatness(y=y, hop_length=512)
    zero_crossing = librosa.feature.zero_crossing_rate(y, hop_length=512)
    #mel = librosa.feature.melspectrogram(y=y, sr=sr, hop_length=512)
    res = np.concatenate((mfcc,spectral_center, chroma, spectral_contrast, spectral_bandwidth, flatness, zero_crossing), 0)
    res = np.mean(res, axis=1)
    return res

function to process all the sound files in a folder and return a numpy array

In [0]:
def folder2features(folder):
    x = []
    y = []
    for p in folder.ls():
        tag = str(p).split("/")[-1]
        print(tag, end = " ")
        for soundfile in p.ls():
            x.append(sound2features(str(soundfile)))
            y.append(d[tag])
    x = np.array(x)
    y = np.array(y)
    return x, y

generating the train and val features

In [17]:
#x_train, y_train = folder2features(Path('meld/train/'))

fear sad neutral disgust happy 

In [21]:
#x_val, y_val = folder2features(Path('meld/val/'))

fear sad neutral disgust happy 

saving the generated features as pickle files to load them later

In [0]:
#with open('xtrain.pkl','wb') as f: pickle.dump(x_train, f)
#with open('ytrain.pkl','wb') as f: pickle.dump(y_train, f)
#with open('xval.pkl','wb') as f: pickle.dump(x_val, f)
#with open('yval.pkl','wb') as f: pickle.dump(y_val, f)

In [0]:
with open('xtrain.pkl','rb') as f: x_train = pickle.load(f)
with open('ytrain.pkl','rb') as f: y_train = pickle.load(f)
with open('xval.pkl','rb') as f: x_val = pickle.load(f)
with open('yval.pkl','rb') as f: y_val = pickle.load(f)

here we see that the dataset has a very high class imbalance

In [386]:
print("Train :", sorted(Counter(y_train).items()))
print("Val   :", sorted(Counter(y_val).items()))

Train : [(0, 4592), (1, 1609), (2, 232), (3, 705), (4, 216)]
Val   : [(0, 517), (1, 181), (2, 28), (3, 79), (4, 25)]


we normalise the dataset to bring it into a respectable range for processing by ml algorithms

In [0]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

since we generated all possible features from the librosa library, so here we sort out the most relevant 40 features

In [0]:
sel = SelectKBest(f_classif, k=40)
xt = sel.fit_transform(x_train, y_train)
xv = sel.transform(x_val)

OneVsRest classifier is the goto classifier when dealing with multilabel classification as it trains individual classifiers for each class with respect to the rest of dataset

In [411]:
clf = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=3))
clf.fit(xt, y_train)
y_pred=clf.predict(xt)
print("Train accuracy    :",accuracy_score(y_true=y_train, y_pred=y_pred))

Train accuracy    : 0.727223279847702


In [412]:
confusion_matrix(y_train, y_pred)

array([[4013,  166,   69,  271,   73],
       [ 532,  721,   68,  232,   56],
       [  73,   13,   95,   39,   12],
       [ 248,   26,    4,  393,   34],
       [  75,   10,    0,    5,  126]])

In [413]:
y_pred=clf.predict(xv)
print("Val accuracy      :",accuracy_score(y_true=y_val, y_pred=y_pred))

Val accuracy      : 0.4566265060240964


In [414]:
confusion_matrix(y_val, y_pred)

array([[342,  55,  12,  80,  28],
       [118,  23,   7,  25,   8],
       [ 13,   6,   1,   5,   3],
       [ 52,   8,   2,  12,   5],
       [ 19,   4,   0,   1,   1]])